In [17]:
from vivit import run_experiment, test_files, classes
from data import get_video_label, VideoDataGenerator, labels_df, files, labels, num_to_label
from synthetic import *
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import numpy as np
import keras
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [64]:
def calculate_motion(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    motion_scores = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Frame differencing
        diff = cv2.absdiff(prev_gray, gray)
        score = np.mean(diff)  # Sum of pixel differences
        motion_scores.append(score)
        
        prev_frame = frame

    cap.release()
    return np.mean(motion_scores)

In [67]:
motion = []
video = 0
for i in files:
    motion.append(calculate_motion(i))
    video += 1
    print(f"{video=}/{len(files)}", end="\r")
labels = [get_video_label(f) for f in files]

In [70]:
motion_df = pd.DataFrame({"motion": motion, "label": labels})
motion_df['class'] = motion_df['label'].apply(lambda x: num_to_label[x])
motion_df = motion_df[['class', 'motion']]
classes = pd.DataFrame(labels_df.groupby('label').size().sort_values(ascending=False))

In [82]:
motions = pd.merge(motion_df, classes, left_on='class', right_index=True)
motions.columns = ['class', 'motion', 'count']
motions = motions.set_index('class').reset_index()
motions = motions.groupby('class')["motion"].agg(['mean', 'std', 'count']).reset_index()

In [93]:
top = motions.sort_values("count", ascending=False).head(10)
right = [0.23076923076923078, 0.775, 0.0, 0.4230769230769231, 0.64, 0.3, 0.13043478260869565, 0.6, 0.46153846153846156, 0.48]
top["right"] = right
import plotly.express as px
px.scatter_3d(top, x='mean', y='std', z='right', color='class', size='count', title='Motion vs. Class')

In [6]:
classes.head(20)

,0
label,
book,40
drink,35
computer,30
before,26
go,26
chair,26
who,25
clothes,25
candy,24


In [ ]:
good_class = ['book', "drink", "go", "chair", "who", "clothes", ""]